In [1]:
#importing necessary libraries
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")

In [2]:
#reading train dataframe
train_df = pd.read_csv('train_target.csv')

In [ ]:
#reading weather dataframe
weather_df = pd.read_csv('weather_2004_v2.csv')

In [4]:
#reading allstation weather dataframe
allstation_df = pd.read_csv('AllStationsData_PHD.txt',sep = '|')

In [5]:
train_df.head()

,FlightNumber,DayOfWeek,Origin,Destination,Distance,ActualArrivalTimeStamp,ScheduledArrTime_c,FlightDelayStatus
0,SMF_ATL_37955,3,SMF,ATL,2092,2004-01-01 19:11:00,2004-01-01 19:11:00,2
1,VLD_ATL_48545,3,VLD,ATL,208,2004-01-01 14:55:00,2004-01-01 14:55:00,2
2,DTW_BOS_31626,3,DTW,BOS,632,2004-01-01 18:33:00,2004-01-01 16:57:00,1
3,MKE_BWI_39603,3,MKE,BWI,641,2004-01-01 15:25:00,2004-01-01 15:25:00,2
4,DFW_CLT_33648,3,DFW,CLT,936,2004-01-01 18:14:00,2004-01-01 18:14:00,2


In [6]:
allstation_df.head()

,WeatherStationID,AirportID,GroundHeight,StationHeight,BarometerHeight,Latitude,Longitude,TimeZone
0,3011,TEX,0,0,0,37.57,-107.55,+7
1,3012,SKX,0,0,0,36.28,-105.40,+7
2,3013,LAA,3683,3703,3675,38.04,-102.41,+7
3,3014,4SL,0,6909,0,35.47,-107.14,+7
4,3016,RIL,5503,5544,5506,39.32,-107.44,+7


In [7]:
weather_df.head()

,WeatherStationID,SkyConditions,Visibility,DBT,RelativeHumidityPercent,WindSpeed,WindDirection,StationPressure,HourlyPrecip,YearMonthDay_Time
0,3011,FEW075 BKN090,10.00,32.0,40.0,15.0,160,30.05,0.0,2004-01-01 13:50:00
1,3011,BKN028 OVC033,10.00,21.0,74.0,5.0,30,30.06,0.0,2004-01-01 19:10:00
2,3011,FEW025 OVC032,9.00,25.0,75.0,4.0,30,30.06,0.0,2004-01-01 20:10:00
3,3011,FEW045 BKN060 OVC080,10.00,34.0,54.0,16.0,240,29.92,0.0,2004-01-02 04:10:00
4,3011,BKN010 OVC017,1.75,25.0,85.0,0.0,0,29.95,0.0,2004-01-02 07:30:00


In [8]:
allstation_df.nunique()

WeatherStationID    1744
AirportID           1744
GroundHeight         720
StationHeight        919
BarometerHeight      730
Latitude            1022
Longitude           1362
TimeZone              12
dtype: int64

In [9]:
weather_df.nunique()

WeatherStationID             1447
SkyConditions              247019
Visibility                     71
DBT                           170
RelativeHumidityPercent        99
WindSpeed                      69
WindDirection                  82
StationPressure              1006
HourlyPrecip                  256
YearMonthDay_Time          218044
dtype: int64

In [14]:
print(train_df.shape)
print(allstation_df.shape)
print(weather_df.shape)

(7861, 8)
(1744, 8)
(6485177, 10)


In [15]:
#checking the datatypes to maintain consistency
def data(*args):
    for i in args:
        print(i.dtypes)
        print()
        
data(train_df,allstation_df,weather_df)

FlightNumber              object
DayOfWeek                  int64
Origin                    object
Destination               object
Distance                   int64
ActualArrivalTimeStamp    object
ScheduledArrTime_c        object
FlightDelayStatus          int64
dtype: object

WeatherStationID      int64
AirportID            object
GroundHeight          int64
StationHeight         int64
BarometerHeight       int64
Latitude            float64
Longitude           float64
TimeZone             object
dtype: object

WeatherStationID             int64
SkyConditions               object
Visibility                 float64
DBT                        float64
RelativeHumidityPercent    float64
WindSpeed                  float64
WindDirection                int64
StationPressure            float64
HourlyPrecip               float64
YearMonthDay_Time           object
dtype: object



In [16]:
#defining a function to check null values of all dataframes at single run
def data(*args):
    for i in args:
        print(i.isna().sum())
        print()
        
data(train_df,allstation_df,weather_df)

FlightNumber              0
DayOfWeek                 0
Origin                    0
Destination               0
Distance                  0
ActualArrivalTimeStamp    0
ScheduledArrTime_c        0
FlightDelayStatus         0
dtype: int64

WeatherStationID    0
AirportID           0
GroundHeight        0
StationHeight       0
BarometerHeight     0
Latitude            0
Longitude           0
TimeZone            0
dtype: int64

WeatherStationID           0
SkyConditions              0
Visibility                 0
DBT                        0
RelativeHumidityPercent    0
WindSpeed                  0
WindDirection              0
StationPressure            0
HourlyPrecip               0
YearMonthDay_Time          0
dtype: int64



In [17]:
# merged weather and all station data frames
weather_2004 = pd.merge(weather_df, allstation_df,how = 'left',on = 'WeatherStationID')

In [18]:
weather_2004.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6485177 entries, 0 to 6485176
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   WeatherStationID         int64  
 1   SkyConditions            object 
 2   Visibility               float64
 3   DBT                      float64
 4   RelativeHumidityPercent  float64
 5   WindSpeed                float64
 6   WindDirection            int64  
 7   StationPressure          float64
 8   HourlyPrecip             float64
 9   YearMonthDay_Time        object 
 10  AirportID                object 
 11  GroundHeight             float64
 12  StationHeight            float64
 13  BarometerHeight          float64
 14  Latitude                 float64
 15  Longitude                float64
 16  TimeZone                 object 
dtypes: float64(11), int64(2), object(4)
memory usage: 890.6+ MB


In [19]:
weather_2004.head()

,WeatherStationID,SkyConditions,Visibility,DBT,RelativeHumidityPercent,WindSpeed,WindDirection,StationPressure,HourlyPrecip,YearMonthDay_Time,AirportID,GroundHeight,StationHeight,BarometerHeight,Latitude,Longitude,TimeZone
0,3011,FEW075 BKN090,10.00,32.0,40.0,15.0,160,30.05,0.0,2004-01-01 13:50:00,TEX,0.0,0.0,0.0,37.57,-107.55,+7
1,3011,BKN028 OVC033,10.00,21.0,74.0,5.0,30,30.06,0.0,2004-01-01 19:10:00,TEX,0.0,0.0,0.0,37.57,-107.55,+7
2,3011,FEW025 OVC032,9.00,25.0,75.0,4.0,30,30.06,0.0,2004-01-01 20:10:00,TEX,0.0,0.0,0.0,37.57,-107.55,+7
3,3011,FEW045 BKN060 OVC080,10.00,34.0,54.0,16.0,240,29.92,0.0,2004-01-02 04:10:00,TEX,0.0,0.0,0.0,37.57,-107.55,+7
4,3011,BKN010 OVC017,1.75,25.0,85.0,0.0,0,29.95,0.0,2004-01-02 07:30:00,TEX,0.0,0.0,0.0,37.57,-107.55,+7


In [20]:
weather_2004.shape

(6485177, 17)

In [21]:
weather_2004.dtypes

WeatherStationID             int64
SkyConditions               object
Visibility                 float64
DBT                        float64
RelativeHumidityPercent    float64
WindSpeed                  float64
WindDirection                int64
StationPressure            float64
HourlyPrecip               float64
YearMonthDay_Time           object
AirportID                   object
GroundHeight               float64
StationHeight              float64
BarometerHeight            float64
Latitude                   float64
Longitude                  float64
TimeZone                    object
dtype: object

In [22]:
# Corelation plot 
corr = weather_2004.corr()
corr.style.background_gradient(cmap='flare').set_precision(2)

,WeatherStationID,Visibility,DBT,RelativeHumidityPercent,WindSpeed,WindDirection,StationPressure,HourlyPrecip,GroundHeight,StationHeight,BarometerHeight,Latitude,Longitude
WeatherStationID,1.00,0.00,0.09,0.00,-0.07,-0.04,-0.07,0.01,0.06,0.06,0.06,-0.01,0.03
Visibility,0.00,1.00,0.02,-0.09,0.02,0.03,-0.01,-0.03,0.01,0.01,0.01,-0.01,0.00
DBT,0.09,0.02,1.00,-0.26,0.03,-0.01,0.05,0.02,-0.09,-0.10,-0.09,-0.52,0.24
RelativeHumidityPercent,0.00,-0.09,-0.26,1.00,-0.23,-0.17,0.16,0.11,-0.21,-0.22,-0.21,0.04,0.11
WindSpeed,-0.07,0.02,0.03,-0.23,1.00,0.45,-0.08,0.04,0.06,0.06,0.06,0.04,-0.06
WindDirection,-0.04,0.03,-0.01,-0.17,0.45,1.00,-0.04,-0.01,0.03,0.03,0.03,0.05,-0.04
StationPressure,-0.07,-0.01,0.05,0.16,-0.08,-0.04,1.00,-0.01,-0.70,-0.66,-0.70,-0.03,0.12
HourlyPrecip,0.01,-0.03,0.02,0.11,0.04,-0.01,-0.01,1.00,0.00,-0.00,0.00,-0.02,0.02
GroundHeight,0.06,0.01,-0.09,-0.21,0.06,0.03,-0.70,0.00,1.00,0.94,1.00,0.02,-0.15
StationHeight,0.06,0.01,-0.10,-0.22,0.06,0.03,-0.66,-0.00,0.94,1.00,0.94,-0.00,-0.16


In [23]:
weather_2004.drop(['GroundHeight','BarometerHeight'],axis = 1,inplace = True)

In [24]:
weather_2004.head()

,WeatherStationID,SkyConditions,Visibility,DBT,RelativeHumidityPercent,WindSpeed,WindDirection,StationPressure,HourlyPrecip,YearMonthDay_Time,AirportID,StationHeight,Latitude,Longitude,TimeZone
0,3011,FEW075 BKN090,10.00,32.0,40.0,15.0,160,30.05,0.0,2004-01-01 13:50:00,TEX,0.0,37.57,-107.55,+7
1,3011,BKN028 OVC033,10.00,21.0,74.0,5.0,30,30.06,0.0,2004-01-01 19:10:00,TEX,0.0,37.57,-107.55,+7
2,3011,FEW025 OVC032,9.00,25.0,75.0,4.0,30,30.06,0.0,2004-01-01 20:10:00,TEX,0.0,37.57,-107.55,+7
3,3011,FEW045 BKN060 OVC080,10.00,34.0,54.0,16.0,240,29.92,0.0,2004-01-02 04:10:00,TEX,0.0,37.57,-107.55,+7
4,3011,BKN010 OVC017,1.75,25.0,85.0,0.0,0,29.95,0.0,2004-01-02 07:30:00,TEX,0.0,37.57,-107.55,+7


In [25]:
weather_2004.tail()

,WeatherStationID,SkyConditions,Visibility,DBT,RelativeHumidityPercent,WindSpeed,WindDirection,StationPressure,HourlyPrecip,YearMonthDay_Time,AirportID,StationHeight,Latitude,Longitude,TimeZone
6485172,94999,CLR,10.0,36.0,64.0,9.0,200,29.84,0.0,2004-11-30 12:55:00,AIT,1204.0,46.33,-93.41,+6
6485173,94999,CLR,10.0,37.0,62.0,6.0,230,29.82,0.0,2004-11-30 13:55:00,AIT,1204.0,46.33,-93.41,+6
6485174,94999,CLR,10.0,37.0,57.0,8.0,220,29.80,0.0,2004-11-30 14:55:00,AIT,1204.0,46.33,-93.41,+6
6485175,94999,CLR,10.0,37.0,62.0,7.0,200,29.79,0.0,2004-11-30 15:55:00,AIT,1204.0,46.33,-93.41,+6
6485176,94999,CLR,10.0,34.0,64.0,4.0,200,29.79,0.0,2004-11-30 16:55:00,AIT,1204.0,46.33,-93.41,+6


In [26]:
weather_2004.dtypes

WeatherStationID             int64
SkyConditions               object
Visibility                 float64
DBT                        float64
RelativeHumidityPercent    float64
WindSpeed                  float64
WindDirection                int64
StationPressure            float64
HourlyPrecip               float64
YearMonthDay_Time           object
AirportID                   object
StationHeight              float64
Latitude                   float64
Longitude                  float64
TimeZone                    object
dtype: object

In [27]:
weather_2004.shape

(6485177, 15)

In [28]:
weather_2004.isna().sum()

WeatherStationID             0
SkyConditions                0
Visibility                   0
DBT                          0
RelativeHumidityPercent      0
WindSpeed                    0
WindDirection                0
StationPressure              0
HourlyPrecip                 0
YearMonthDay_Time            0
AirportID                  928
StationHeight              928
Latitude                   928
Longitude                  928
TimeZone                   928
dtype: int64

In [33]:
weather_2004['TimeZone'].nunique()

11

In [34]:
weather_2004['TimeZone'].unique()

array(['+7', '+6', '+8', '+5', '+4', '+10', '+9', 'Yukon Standard',
       'Bering Standard', '-10', 'Alaska-Hawaii', nan], dtype=object)

In [37]:
#creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing
weather_2004['TimeZone'] = labelencoder.fit_transform(weather_2004['TimeZone'])

In [38]:
weather_2004['TimeZone'].unique()

array([ 4,  3,  5,  2,  1,  0,  6, 10,  9,  7,  8, 11])

In [39]:
# Final consolidated weather data
weather_2004.head()

,WeatherStationID,SkyConditions,Visibility,DBT,RelativeHumidityPercent,WindSpeed,WindDirection,StationPressure,HourlyPrecip,YearMonthDay_Time,AirportID,StationHeight,Latitude,Longitude,TimeZone
0,3011,FEW075 BKN090,10.00,32.0,40.0,15.0,160,30.05,0.0,2004-01-01 13:50:00,TEX,0.0,37.57,-107.55,4
1,3011,BKN028 OVC033,10.00,21.0,74.0,5.0,30,30.06,0.0,2004-01-01 19:10:00,TEX,0.0,37.57,-107.55,4
2,3011,FEW025 OVC032,9.00,25.0,75.0,4.0,30,30.06,0.0,2004-01-01 20:10:00,TEX,0.0,37.57,-107.55,4
3,3011,FEW045 BKN060 OVC080,10.00,34.0,54.0,16.0,240,29.92,0.0,2004-01-02 04:10:00,TEX,0.0,37.57,-107.55,4
4,3011,BKN010 OVC017,1.75,25.0,85.0,0.0,0,29.95,0.0,2004-01-02 07:30:00,TEX,0.0,37.57,-107.55,4


In [40]:
weather_2004['SkyConditions'].nunique()

247019

In [41]:
#saving the dataset in csv format
weather_2004.to_csv('final_weather_data.csv',index_label = False)